In [9]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
import os
from glob import glob
import cv2


In [ ]:
# 1. Data Loading and Preprocessing
# Define data directories
data_dir = "C:Data-Engineering/ai-images-health/cow-images/trained"
image_size = (224, 224)  # ResNet input image size
batch_size = 32

In [22]:
# Load images and labels
def load_images_and_labels(data_dir):
    images = []
    labels = []
    for label, class_dir in enumerate(["healthy", "unhealthy"]):  # Define folder names
        class_path = os.path.join(data_dir, class_dir)
        for image_path in glob(os.path.join(class_path, "*.jpg")):  # Assuming .jpg format
            img = cv2.imread(image_path)
            img = cv2.resize(img, image_size)  # Resize to 224x224
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

X, y = load_images_and_labels(data_dir)


In [23]:
# Normalize images
X = X / 255.0

In [24]:
# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# 2. Transfer Learning - ResNet50 Model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)  # Binary classification: healthy or unhealthy

# Define model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss="binary_crossentropy", 
              metrics=["accuracy"])

In [ ]:
# 3. Train the Model
print("Training the model...")
history = model.fit(X_train, y_train, 
                    validation_split=0.1, 
                    epochs=10, 
                    batch_size=batch_size)


In [ ]:
# 4. Evaluate the Model
print("Evaluating the model...")
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


In [18]:
def preprocess_image(image_path, image_size=(224, 224)):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Error: Image not loaded. Check the file path.")
    img = cv2.resize(img, image_size)  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img


In [60]:
def predict_image(model, image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    
    # Get prediction
    prediction = model.predict(img)
    label = "unhealthy" if prediction[0][0] > 0.39 else "healthy"
    confidence = prediction[0][0] if label == "unhealthy" else 1 - prediction[0][0]
    
    print(f"Prediction: {label} (Confidence: {confidence:.2f})")

In [ ]:
image_path = "C:/Data-Engineering/ai-images-health/cow-images/predict/u-cow-3.jpeg"
predict_image(model, image_path)
